In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%config InlineBackend.figure_format = 'retina'

In [2]:
from scripts.convert_brat_annotations_to_json import *
import seaborn as sns
import matplotlib.pyplot as plt
from baseline.baseline import *
from scripts.analyse_pwc_entity_results import *

In [3]:
df_concat = combine_brat_to_original_data('../data/pwc_s2_cleaned_text_v2.jsonl',
                                         '../data/pwc_s2_cleaned_text_v2_sentences.jsonl',
                                         '../outputs/pwc_s2_cleaned_text_v2_sentences_predictions.jsonl',
                                         '../brat/data/result_extraction/outputs/brat_annotation_folder_doclevel_original/',
                                         '../brat/data/result_extraction/outputs/brat_annotation_folder_doclevel/')

314630it [01:07, 4643.10it/s]


314630 314630


  5%|▍         | 53/1172 [00:00<00:21, 52.47it/s]

../brat/data/result_extraction/outputs/brat_annotation_folder_doclevel/.stats_cache  have not document


 58%|█████▊    | 681/1172 [00:06<00:05, 93.67it/s] 

...

 92%|█████████▏| 1075/1172 [00:11<00:00, 153.37it/s]

Attribute before Trigger


100%|██████████| 1172/1172 [00:12<00:00, 96.18it/s] 


../brat/data/result_extraction/outputs/brat_annotation_folder_doclevel_original/not_found  have not document


100%|██████████| 1170/1170 [00:03<00:00, 356.26it/s]


In [4]:
from dygie.data.dataset_readers.read_pwc_dataset import *

In [5]:
data = read_dataframe(df_concat)
dump_to_file(data, '../data/pwc_split_on_labeled', max_id='0e37c8f19eefeb0c20d92f5cb4df4153077c116b')

100%|██████████| 1170/1170 [00:41<00:00, 28.08it/s]


In [ ]:
from tabulate import tabulate

In [ ]:
df = pd.DataFrame(df_concat[df_concat.clean_type != 'normal'].groupby(['doc_id', 'para_num'])['words']\
                        .apply(lambda x : sum([len(w) for w in x]))\
                        .describe(percentiles=[0.5, 0.75, 0.9, 0.99])).transpose()
print(tabulate(df.values, headers=df.columns, tablefmt="github"))

In [ ]:
df_concat.groupby('doc_id')['words']\
.agg(lambda x : "our system" in " ".join([w.lower() for y in x for w in y])).sum()

Statistics 
===========

In [ ]:
df_done = df_concat[df_concat.doc_id < '0e37c8f19eefeb0c20d92f5cb4df4153077c116b']
print(len(df_done.doc_id.unique()))
n_spans_entity = sum([x['type_change'] for x in list(df_done['stats']) if x is not None])
n_spans_attr = sum([x['change_attributes'] for x in list(df_done['stats']) if x is not None])
n_spans_kept = sum([x['spans_kept'] for x in list(df_done['stats']) if x is not None])
n_spans_old = sum([x['old_spans'] for x in list(df_done['stats']) if x is not None])
print(n_spans_entity, n_spans_attr, n_spans_kept, n_spans_old)

In [ ]:
def get_linked_entities_from_span(spans) :
    index_map = {k:set() for k in used_entities}
    for span in spans :
        if len(span.links) > 0 :
            index_map[span.entity] |= span.links
    return index_map
            
df_done = df_concat[df_concat.doc_id < '0e37c8f19eefeb0c20d92f5cb4df4153077c116b']
linked_entities = df_done['entities'].progress_apply(get_linked_entities_from_span)
df_linked = pd.concat([df_done, pd.DataFrame.from_records(linked_entities)], axis=1)

In [ ]:
true_elist = df_linked.groupby('doc_id')[[k + '_Rel' for k in used_entities]].agg(lambda x : set(x.iloc[0].keys()))

In [ ]:
ann_elist = df_linked.groupby('doc_id')[used_entities].agg(lambda x : set.union(*x))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = [x for y in axes for x in y]
for i, e in enumerate(used_entities) :
    true_ent = true_elist[e + '_Rel']
    pred_ent = ann_elist[e]
    nent = [len(x - y) for x, y in zip(true_ent, pred_ent)]
    sns.countplot(nent, ax=axes[i])
    axes[i].set_title(e)
plt.tight_layout()
plt.show()

In [ ]:
para_aggregating_function = {t:lambda x : set().union(*x) for t in used_entities}
para_aggregating_function.update({'Relations':lambda x : x.values[-1]})
df_linked_para = df_linked.groupby(['doc_id', 'para_num'])[used_entities + ['Relations']].agg(para_aggregating_function)

In [ ]:
from itertools import combinations, product
binary_relations = list(combinations(used_entities, 2))

In [ ]:
def generate_list_of_binary_relations(rels) :
    index_map = {k:[] for k in binary_relations}
    for r in rels :
        r = r._asdict()
        for k in binary_relations :
            index_map[k].append((r[k[0]], r[k[1]]))
            
    return {k:set(v) for k, v in index_map.items()}

df_linked_para['binary_relations_true'] = df_linked_para['Relations'].progress_apply(lambda x : generate_list_of_binary_relations(x))

In [ ]:
def get_relation_para_mapping(rows) :
    true_relations = rows['binary_relations_true'].values[-1]
    relation_paras = {k:{x:0 for x in list(v)} for k, v in true_relations.items()}
    for index, row in rows.iterrows() :
        for r1, r2 in binary_relations :
            rels = product(row[r1], row[r2])
            for r in rels :
                if r in relation_paras[(r1, r2)] :
                    relation_paras[(r1, r2)][r] += 1
    return relation_paras
                
df_linked_para_stats = df_linked_para.groupby('doc_id').progress_apply(get_relation_para_mapping)
df_linked_para_stats = df_linked_para_stats.apply(lambda x : pd.Series({"-".join(k):v for k, v in x.items()}))
fig, axes = plt.subplots(3, 2, figsize=(10, 10))
axes = [x for y in axes for x in y]
df_linked_para_stats_done = df_linked_para_stats[df_linked_para_stats.index < '0e37c8f19eefeb0c20d92f5cb4df4153077c116b']
for i, r in enumerate(binary_relations) :
    sns.countplot(df_linked_para_stats_done.applymap(lambda x : list(x.values()))\
                  .apply(lambda x : [v for y in x for v in y])['-'.join(r)], ax=axes[i])
    axes[i].set_title(r)
plt.tight_layout()

In [ ]:
df_linked['linked_spans'] = df_linked['entities'].apply(lambda cluster : sum([len(x.links) > 0 for x in cluster]))
fig = plt.figure()
ax = plt.gca()
for n in range(2) :
    stats = (df_linked.groupby(['doc_id', 'para_num'])['linked_spans'].sum() > n).reset_index()\
                      .groupby(['doc_id'])['linked_spans'].agg([('linked', 'sum'), ('total', len)])

    sns.regplot(x='total', y='linked', data=stats[stats.index < '0e37c8f19eefeb0c20d92f5cb4df4153077c116b'], label=n, scatter_kws={'s' : 5})
    plt.legend(title="n")
    plt.xlabel("Total #paragraphs")
    plt.ylabel("#paragraphs with $>n$ linked entities")

In [ ]:
def get_relation_para_mapping(row) :
    true_relations = row['binary_relations_true']
    contain_rel = 0
    for r1, r2 in binary_relations :
        rels = product(row[r1], row[r2])
        for r in rels :
            if r in true_relations[(r1, r2)] :
                contain_rel += 1
    return contain_rel
                
df_num_rel_para_stats = df_linked_para.progress_apply(get_relation_para_mapping, axis=1)
stats = df_num_rel_para_stats.reset_index().groupby(['doc_id'])[0].agg([('linked', lambda x: sum([y>0 for y in x])), ('total', len)])

sns.lmplot(x='total', y='linked', data=stats[stats.index < '0e37c8f19eefeb0c20d92f5cb4df4153077c116b'])
plt.legend()

In [ ]:
from collections import Counter
def count_and_clip(sets) :
    c = sum([Counter(x) for x in sets], Counter())
    c = Counter({k:(1 if v > 0 else 0) for k, v in c.items()})
    return [c]

def para_with_one_entity(counters) :
    c = counters[0]
    return sum([v <= 1 for k, v in c.items()])

para_counts = df_linked.groupby(['doc_id', 'section_id', 'para_id'])['Material'].progress_apply(count_and_clip)
stats = para_counts.groupby('doc_id').agg([('counts', lambda x : para_with_one_entity([sum([w[0] for w in x], Counter())]))
                                                    , ('total', len)])

sns.lmplot(x='total', y='counts', data=stats[stats.index < '07cca2bdd0dc2fee02889e17789748eba9d06ffa'])
plt.legend()

In [ ]:
for t in enttypes :
    df_done = df_linked[df_linked.doc_id < '09da677bdbba113374d8fe4bb15ecfbdb4c8fe40']
    true_ent = df_done.groupby(['doc_id'])[t + '_Rel'].apply(lambda x: set.union(*[set(w.keys()) for w in x]))
    ann_links = df_done.groupby(['doc_id'])[t].apply(lambda x : set.union(*list(x)))
    link_sets = pd.concat([true_ent, ann_links], axis=1)
    
    print(t, sum([len(x - y) > 0 for x, y in zip(true_ent, ann_links)]), sum([len(y - x) > 0 for x, y in zip(true_ent, ann_links)]))

Document Statistics
===================

In [ ]:
from tabulate import tabulate

In [ ]:
df = pd.DataFrame(df_concat.groupby('doc_id')['words']\
                        .apply(lambda x : sum([len(w) for w in x]))\
                        .describe(percentiles=[0.5, 0.75, 0.9, 0.99])).transpose()
print(tabulate(df.values, headers=df.columns, tablefmt="github"))

In [ ]:
df = pd.DataFrame(df_concat.groupby('doc_id')['sentence']\
                        .apply(lambda x : sum([len(w) for w in x]))\
                        .describe(percentiles=[0.5, 0.75, 0.9, 0.99])).transpose()
print(tabulate(df.values, headers=df.columns, tablefmt="github"))

In [ ]:
df = pd.DataFrame(df_concat.groupby(['doc_id', 'para_num'])['words']\
                        .apply(lambda x : sum([len(w) for w in x]))\
                        .describe(percentiles=[0.5, 0.75, 0.9, 0.99])).transpose()
print(tabulate(df.values, headers=df.columns, tablefmt="github"))

In [ ]:
df = pd.DataFrame(df_concat.groupby(['doc_id'])['para_num'].agg(lambda x: max(x) + 1)\
                        .describe(percentiles=[0.5, 0.75, 0.9, 0.99])).transpose()
print(tabulate(df.values, headers=df.columns, tablefmt="github"))

In [ ]:
from scipy import stats
stats.linregress(para_features.groupby('doc_id')['entities'].agg(len), para_features.groupby('doc_id')['entities'].agg(sum))